### Fuzzy String Matcher (Basic)

In [205]:
import pandas as pd
import numpy as np
from string_grouper import match_strings, match_most_similar, \
	group_similar_strings, compute_pairwise_similarities, \
	StringGrouper
import cleanco

ModuleNotFoundError: No module named 'string_grouper'

In [ ]:
## directory info
rootDataPath = 'INSERT YOUR FILEPATH HERE'

In [ ]:
## file read
matchlist = pd.read_csv(f'{rootDataPath}company_matches_pre_cleaning.csv')

In [ ]:
## establish reference list for all matches to be matched against
ref_list = matchlist[['companyId','companyName']].drop_duplicates().copy()

## remove the following words from the company names
pattern = '|'.join(['-','_','technologies','technology','pharmaceuticals','partners', 'international',
'financial services','management','consulting','industries','entertainment', 'enterprises','com'
'networks','life sciences','capital markets','communications','resources','healthcare','therapeutics','health care'])

## replace those words
ref_list['dehyphenName'] = ref_list.companyName.str.replace(pattern,' ')

## apply company name cleaning
ref_list['cleanMain'] = ref_list.apply(lambda x: cleanco.basename(x.dehyphenName), axis=1)

In [ ]:
## establish list of companies to be matched (the reference list is where we'll be matching them)
to_match_list = matchlist[['companyName2']].drop_duplicates().copy().astype(str)
to_match_list['dehyphenName'] = to_match_list.companyName2.str.replace(pattern, ' ')
to_match_list['cleanToMatch'] = to_match_list.apply(lambda x: cleanco.basename(x.dehyphenName), axis=1)

In [ ]:
## break apart the match list to speed things up into blocks of 600k
matchlists_chunks = [to_match_list.cleanToMatch[0:600000].copy(), to_match_list.cleanToMatch[600000:1200000].copy(), to_match_list.cleanToMatch[1200000:].copy()]
matchlists_chunks = [item.astype(str) for item in matchlists_chunks]

In [ ]:
## test to check similarity threshold. going with 0.67 for now.
matches1 = match_strings(ref_list.cleanMain, matchlists_chunks[0], n_blocks='auto', min_similarity=0.67)
matches1.sort_values(by='similarity', ascending=True).head(20)

In [ ]:
## concatenate all of the matches
matches = pd.concat([match_strings(ref_list.cleanMain, item, n_blocks='auto', min_similarity=0.67) for item in matchlists_chunks], axis=0, ignore_index=True)
matches.sort_values(by='similarity', ascending=False)

In [148]:
def retrieveData():
    url = "https://www.sec.gov/files/company_tickers.json"
    response = requests.get(url)
    json_data = response.json()
    file_name = "company_tickers.json"
    with open(file_name, "w") as json_file:
        json.dump(json_data, json_file)
    print(f"JSON data saved to {file_name}")

def loadData():
    file_name = "company_tickers.json"
    with open(file_name, "r") as json_file:
        loaded_json_data = json.load(json_file)
    df = pd.DataFrame(loaded_json_data)
    df = df.transpose()
    df.title = df.title.str.title()
    df = df.rename({"title":"Company"},axis=1)
#     df = df[~df.ticker.str.contains('-')]
#     df = df.drop_duplicates(subset='Company', keep='first')
    df.to_csv("SEC-CompanyTicker.csv")
    return df
df = loadData()


In [192]:
from sentence_transformers import SentenceTransformer
import faiss
import requests
import json
import pandas as pd
import numpy as np
companyNames = pd.read_csv("SEC-Data.csv")
companyNames = pd.DataFrame({"Company":["Wells Fargo","Sculptor"]}) 


class Similarity:  
    def __init__(self):
        self.possibleCompanies = loadData()

        
    def TrainFaiss(self):
        self.encoder = SentenceTransformer("paraphrase-mpnet-base-v2")
        vectors = self.encoder.encode(list(self.possibleCompanies.Company))
        vector_dimension = vectors.shape[1]
        self.index = faiss.IndexFlatL2(vector_dimension)
        faiss.normalize_L2(vectors)
        self.index.add(vectors)
        
    def getCompanyMatch(self, companyNames):
        possibleCompanies = loadData()
        possibleCompanies = possibleCompanies[~possibleCompanies.ticker.str.contains('-')]
        possibleCompanies = possibleCompanies.drop_duplicates(subset='Company', keep='first')

        matches = pd.DataFrame(columns=["Company",'cik_str', 'ticker', 'Match'])
        for i,company in enumerate(companyNames["Company"]):
            match = predict(company)
            matches.loc[i] = [company] + list(match.iloc[0]) 
        return matches
            
    def predict(self,search_text):
        search_vector = self.encoder.encode(search_text)
        _vector = np.array([search_vector])
        faiss.normalize_L2(_vector)
        k = self.index.ntotal
        distances, ann = self.index.search(_vector, k=k)
        return self.possibleCompanies[ann[0][0]]
    
    

In [193]:

matcher = Similarity()


In [194]:
matcher.TrainFaiss()


In [195]:
matcher.getCompanyMatch(companyNames)

Company  cik_str ticker                              Match
0  Wells Fargo    72971    WFC           Wells Fargo & Company/Mn
1     Sculptor  1403256    SCU  Sculptor Capital Management, Inc.

In [198]:
companyNames = pd.read_csv("SEC-Data.csv")
matcher.getCompanyMatch(companyNames[0:10])

Company  cik_str ticker  \
0                       3Com Corp    66740    MMM   
1                      3M Company    66740    MMM   
2               A.G. Edwards Inc.  1099800     EW   
3             Abbott Laboratories     1800    ABT   
4         Abercrombie & Fitch Co.  1018840    ANF   
5     ABM Industries Incorporated   771497    ABM   
6        Ace Hardware Corporation  1844389   ACBA   
7          ACT Manufacturing Inc.  1388320   ATNM   
8                   Acterna Corp.  1388320   ATNM   
9  Adams Resources & Energy, Inc.     2178     AE   

                                 Match  
0                                3M Co  
1                                3M Co  
2            Edwards Lifesciences Corp  
3                  Abbott Laboratories  
4          Abercrombie & Fitch Co /De/  
5              Abm Industries Inc /De/  
6  Ace Global Business Acquisition Ltd  
7       Actinium Pharmaceuticals, Inc.  
8       Actinium Pharmaceuticals, Inc.  
9       Adams Resources & Energy, Inc.

In [200]:
possibleCompanies = pd.read_csv("SEC-CompanyTicker.csv",index_col=0)
matcher.getCompanyMatch(possibleCompanies[0:100])

Company  cik_str ticker                       Match
0                   Apple Inc.   320193   AAPL                  Apple Inc.
1               Microsoft Corp   789019   MSFT              Microsoft Corp
2                Alphabet Inc.  1652044  GOOGL               Alphabet Inc.
3               Amazon Com Inc  1018724   AMZN              Amazon Com Inc
4                  Nvidia Corp  1045810   NVDA                 Nvidia Corp
..                         ...      ...    ...                         ...
95       Booking Holdings Inc.  1075531   BKNG       Booking Holdings Inc.
96              Starbucks Corp   829224   SBUX              Starbucks Corp
97  Anheuser-Busch Inbev Sa/Nv  1668717    BUD  Anheuser-Busch Inbev Sa/Nv
98       Toronto Dominion Bank   947263     TD       Toronto Dominion Bank
99                    Rtx Corp   101829    RTX                    Rtx Corp

[100 rows x 4 columns]